In [1]:
! pip install xgboost
from imblearn.over_sampling import SMOTE, ADASYN
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fastai.tabular.all import df_shrink
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import make_pipeline
import pickle
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv("/content/TON_IOT_pre.unknown")

In [3]:
df.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack,IPV4_SRC_ADDR_0,IPV4_SRC_ADDR_1,IPV4_SRC_ADDR_2,IPV4_SRC_ADDR_3,IPV4_DST_ADDR_0,IPV4_DST_ADDR_1,IPV4_DST_ADDR_2,IPV4_DST_ADDR_3
0,63318,443,6,91.00,181,165,2,1,24,327,0,Benign,192,168,1,195,52,139,250,253
1,57442,15600,17,0.00,63,0,1,0,0,0,0,Benign,192,168,1,79,192,168,1,255
2,57452,15600,17,0.00,63,0,1,0,0,0,0,Benign,192,168,1,79,239,255,255,250
3,51989,15600,17,0.00,63,0,1,0,0,0,0,Benign,192,168,1,79,192,168,1,255
4,53927,1900,17,153.12,459,0,3,0,0,614,0,Benign,192,168,1,6,239,255,255,250


In [4]:
df.drop(columns='Label', inplace=True)

In [5]:
df_1 = df[df['Attack']!="Benign"]

#SMOT

In [6]:
from imblearn.over_sampling import SMOTE

In [7]:
X = df_1.drop(columns='Attack')
y = df_1['Attack']

In [8]:
oversample = SMOTE()

In [9]:
X, y = oversample.fit_resample(X, y)

In [10]:
print(X.shape)
print(y.shape)

(4179987, 18)
(4179987,)


In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y= label_encoder.fit_transform(y)


In [13]:
print(y.shape)

(4179987,)


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [18]:
ss = StandardScaler()
ss.fit_transform(X_train, X_test)

array([[ 0.53496173, -0.30005756, -0.50321215, ...,  0.20927287,
        -0.26371878,  0.82294287],
       [-1.08254076, -0.30005756, -0.50321215, ...,  0.20927287,
        -0.26371878,  0.82294287],
       [-1.63071249, -0.30005756, -0.50321215, ...,  0.20927287,
        -0.26371878,  0.82294287],
       ...,
       [-1.26790384, -0.30005756, -0.50321215, ...,  0.20927287,
        -0.26371878,  0.35769626],
       [ 0.12138781, -0.30005756, -0.50321215, ...,  0.20927287,
        -0.26371878,  0.88415953],
       [-0.77457257, -0.30005756, -0.50321215, ...,  0.20927287,
        -0.26371878,  0.82294287]])

#Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,Y_train)
test_predictions = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
preds = model.predict(X_test)

In [43]:
print(classification_report(Y_test,preds))

              precision    recall  f1-score   support

           0       0.98      0.58      0.73    139309
           1       0.09      0.06      0.08    139525
           2       0.35      0.99      0.51    139568
           3       0.31      0.53      0.39    138815
           4       0.35      0.28      0.31    139760
           5       0.28      0.31      0.30    139478
           6       0.99      0.74      0.84    139371
           7       0.52      0.18      0.27    139145
           8       0.26      0.04      0.06    139026

    accuracy                           0.41   1253997
   macro avg       0.46      0.41      0.39   1253997
weighted avg       0.46      0.41      0.39   1253997



In [45]:
import pickle
with open('Regression-SMOTE-multi.pkl', 'wb') as f:
    pickle.dump(model, f)

#RandomForestClassifier

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
model = RandomForestClassifier(n_estimators=10,max_features='auto',random_state=101)

In [35]:
model.fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_features='auto', n_estimators=10, random_state=101)

In [36]:
preds = model.predict(X_test)

In [37]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [38]:
print(classification_report(Y_test,preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    139309
           1       0.67      0.63      0.65    139525
           2       0.87      0.92      0.89    139568
           3       0.45      0.42      0.44    138815
           4       0.99      0.99      0.99    139760
           5       0.54      0.53      0.54    139478
           6       1.00      1.00      1.00    139371
           7       0.83      0.86      0.85    139145
           8       0.75      0.79      0.77    139026

    accuracy                           0.79   1253997
   macro avg       0.79      0.79      0.79   1253997
weighted avg       0.79      0.79      0.79   1253997



In [40]:
import pickle
with open('Randomfarest-SMOTE-multi.pkl', 'wb') as f:
    pickle.dump(model, f)

#Catgboost

In [29]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.3 MB/s eta 0:00:00


In [30]:
import catboost as ctb
model =  ctb.CatBoostClassifier()
model.fit(X_train, Y_train)

Learning rate set to 0.118878
0:	learn: 1.7047723	total: 9.3s	remaining: 2h 34m 53s
1:	learn: 1.4901312	total: 17s	remaining: 2h 20m 59s
2:	learn: 1.3372549	total: 27s	remaining: 2h 29m 22s
3:	learn: 1.2271678	total: 36.5s	remaining: 2h 31m 26s
4:	learn: 1.1326462	total: 44s	remaining: 2h 25m 50s
5:	learn: 1.0595375	total: 52.6s	remaining: 2h 25m 11s
6:	learn: 0.9981039	total: 60s	remaining: 2h 21m 48s
7:	learn: 0.9489541	total: 1m 6s	remaining: 2h 18m 16s
8:	learn: 0.9045303	total: 1m 10s	remaining: 2h 9m 6s
9:	learn: 0.8681918	total: 1m 14s	remaining: 2h 2m 11s
10:	learn: 0.8356707	total: 1m 19s	remaining: 1h 58m 29s
11:	learn: 0.8074324	total: 1m 24s	remaining: 1h 55m 22s
12:	learn: 0.7825153	total: 1m 31s	remaining: 1h 56m 18s
13:	learn: 0.7577558	total: 1m 36s	remaining: 1h 53m 2s
14:	learn: 0.7327607	total: 1m 40s	remaining: 1h 49m 45s
15:	learn: 0.7143581	total: 1m 44s	remaining: 1h 47m 35s
16:	learn: 0.6944471	total: 1m 49s	remaining: 1h 45m 5s
17:	learn: 0.6774589	total: 1m 53

In [31]:
y_pred= model.predict(X_test)
score = accuracy_score(Y_test, y_pred)
print("Accuracy:", score)
print(classification_report(Y_test,y_pred))

Accuracy: 0.8357452210810712
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    139309
           1       0.82      0.63      0.71    139525
           2       0.80      0.98      0.88    139568
           3       0.87      0.40      0.55    138815
           4       0.97      0.97      0.97    139760
           5       0.61      0.72      0.66    139478
           6       1.00      1.00      1.00    139371
           7       0.77      0.83      0.80    139145
           8       0.76      0.99      0.86    139026

    accuracy                           0.84   1253997
   macro avg       0.85      0.84      0.83   1253997
weighted avg       0.85      0.84      0.83   1253997



In [32]:
import pickle
with open('Catboost-SMOTE-multi.pkl', 'wb') as f:
    pickle.dump(model, f)